This guide shows you how to set up and use LangGraph's prebuilt, reusable components, which are designed to help you construct agentic systems quickly and reliably.


# 第一步安装依赖

In [ ]:
!pip install -U langgraph

# 2. 创建一个agent

In [3]:
from langchain_community.chat_models import ChatTongyi
llm = ChatTongyi(
    model="qwen-max"
)

In [5]:
from langgraph.prebuilt import create_react_agent

def get_weather(city: str) -> str:  
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_react_agent(
    model=llm,
    tools=[get_weather],  
    prompt="You are a helpful assistant"  
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='e6fbebaa-6d46-4a2e-8428-cd9fe6fe8099'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'name': 'get_weather', 'arguments': '{"city": "sf"}'}, 'index': 0, 'id': 'call_2185a0d660494c68893373', 'type': 'function'}]}, response_metadata={'model_name': 'qwen-max', 'finish_reason': 'tool_calls', 'request_id': '60312dcc-3053-9deb-af3c-25ebc96a7edd', 'token_usage': {'input_tokens': 171, 'output_tokens': 17, 'total_tokens': 188, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--1d186acb-6a97-46e7-89b4-64b7ab204dc3-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'call_2185a0d660494c68893373', 'type': 'tool_call'}]),
  ToolMessage(content="It's always sunny in sf!", name='get_weather', id='8c7113e6-36b9-43cf-8ae2-5c129dfc4f7d', tool_call_id='call_2185a0d660494c68893373'),
  AIMessage(content="It's always sunny in San Francisco (SF)! 

# Add a custom prompt

## 使用静态提示词

In [8]:
from langgraph.prebuilt import create_react_agent

llm = ChatTongyi(
    model="qwen-max"
)

agent = create_react_agent(
    model=llm,
    tools=[get_weather],
    # A static prompt that never changes
    prompt="Never answer questions about the weather."
)

agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='b312ddc6-5191-493b-bdb7-8bed977bd962'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'name': 'get_weather', 'arguments': '{"city": "San Francisco"}'}, 'index': 0, 'id': 'call_0dde97af7f1e4c99b49c1c', 'type': 'function'}]}, response_metadata={'model_name': 'qwen-max', 'finish_reason': 'tool_calls', 'request_id': '0ede0308-335e-9e1d-930e-27c52d935173', 'token_usage': {'input_tokens': 172, 'output_tokens': 18, 'total_tokens': 190, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--df558f9d-36b5-454b-a49f-ff7999c94788-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_0dde97af7f1e4c99b49c1c', 'type': 'tool_call'}]),
  ToolMessage(content="It's always sunny in San Francisco!", name='get_weather', id='725ffa0c-019e-4646-9029-e643af11d370', tool_call_id='call_0dde97af7f1e4c99b49c1c'),
  AIMessage(content="It seems

## 使用动态提示词

In [9]:
from langchain_core.messages import AnyMessage
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.prebuilt import create_react_agent
from langchain.chat_models import ChatTongyi

llm = ChatTongyi(
    model="qwen-max"
)


def prompt(state: AgentState, config: RunnableConfig) -> list[AnyMessage]:  
    user_name = config["configurable"].get("user_name")
    system_msg = f"You are a helpful assistant. Address the user as {user_name}."
    return [{"role": "system", "content": system_msg}] + state["messages"]


agent = create_react_agent(
    model=llm,
    tools=[get_weather],
    prompt=prompt
)

agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    config={"configurable": {"user_name": "John Smith"}}
)



{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='24ef88e9-f84f-4144-be29-1c6fd6a3149c'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'name': 'get_weather', 'arguments': '{"city": "San Francisco, US"}'}, 'index': 0, 'id': 'call_dacc68277e4446ec8d1538', 'type': 'function'}]}, response_metadata={'model_name': 'qwen-max', 'finish_reason': 'tool_calls', 'request_id': '58997d05-a569-9735-a251-eb389941bb7b', 'token_usage': {'input_tokens': 178, 'output_tokens': 20, 'total_tokens': 198, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--792346f3-5f4e-4f2a-a26c-f738f2283f74-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco, US'}, 'id': 'call_dacc68277e4446ec8d1538', 'type': 'tool_call'}]),
  ToolMessage(content="It's always sunny in San Francisco, US!", name='get_weather', id='eed8ffbf-884b-478b-b324-b4997ee5ffac', tool_call_id='call_dacc68277e4446ec8d1538'),
  AIMessage(conte

# 添加记忆

In [11]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

llm = ChatTongyi(
    model="qwen-max"
)

agent = create_react_agent(
    model=llm,
    tools=[get_weather],
    checkpointer=checkpointer  
)

# Run the agent
config = {"configurable": {"thread_id": "1"}}
sf_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    config  
)
ny_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what about new york?"}]},
    config
)

# 结构化的输出


In [16]:
from pydantic import BaseModel
from langgraph.prebuilt import create_react_agent

class WeatherResponse(BaseModel):
    conditions: str

agent = create_react_agent(
    model=llm,
    tools=[get_weather],
    response_format=WeatherResponse  
)

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in shanghai"}]}
)
response

# response["structured_response"]

{'messages': [HumanMessage(content='what is the weather in shanghai', additional_kwargs={}, response_metadata={}, id='f6267c08-f913-4e8a-8082-7dbf94b4fa16'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'name': 'get_weather', 'arguments': '{"city": "shanghai"}'}, 'index': 0, 'id': 'call_4c8113cc8c294df58f8a2f', 'type': 'function'}]}, response_metadata={'model_name': 'qwen-max', 'finish_reason': 'tool_calls', 'request_id': 'ee5c31b0-344a-9760-a2d1-ac09c39cbf70', 'token_usage': {'input_tokens': 167, 'output_tokens': 18, 'total_tokens': 185, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--67793e61-0934-4304-a22d-84f3a856696c-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'shanghai'}, 'id': 'call_4c8113cc8c294df58f8a2f', 'type': 'tool_call'}]),
  ToolMessage(content="It's always sunny in shanghai!", name='get_weather', id='2dd8f9ff-fd06-45aa-a2d7-51f8c7139093', tool_call_id='call_4c8113cc8c294df58f8a2f'),
  AIMessage(content='The weather in Sh